In [37]:
import pandas as pd 
import numpy as np 

data_path = '../Datasets/Data23.xlsx'

sales = pd.read_excel(data_path,sheet_name = 'SALES', parse_dates = ['DateTime'])
products = pd.read_excel(data_path,sheet_name = "PRODUCTS")
customers = pd.read_excel(data_path,sheet_name = 'CUSTOMERS', parse_dates = ['UserFirstTransaction'])

In [38]:
sales.head(3)

,TransactionID,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount
0,1,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.0,No
1,2,500240813,2017-01-01 03:27:58.490,10610,WEB,Cash,108.0,No
2,3,500460527,2017-01-01 04:42:46.500,10579,MOBILE,Cash,22.5,No


In [39]:
customers.head(3)

,UserID,UserFirstTransaction,Gender,Location,Age
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19
1,500234631,2018-04-06 15:19:33.990,FEMALE,BURSA,20
2,500234642,2016-04-23 00:00:00.000,MALE,IZMIR,41


In [40]:
products.head(3)

,ProductID,Category
0,10001,Female Shoes
1,10002,Male Shoes
2,10003,TVs and TV Sets


In [41]:
today = sales['DateTime'].max()

last_transactions = sales.groupby('UserID')['DateTime'].max().reset_index()
last_transactions.columns = ['UserID','LastTransaction']
last_transactions['Recency'] = (today - last_transactions['LastTransaction']).dt.days
last_transactions['Churned'] = np.where(last_transactions['Recency'] > 252, 1,0)
last_transactions.head()

,UserID,LastTransaction,Recency,Churned
0,500234532,2017-12-17 18:08:28.150,195,0
1,500234631,2018-05-11 13:22:46.950,50,0
2,500234642,2018-06-17 19:15:22.400,13,0
3,500234730,2017-01-29 18:49:51.470,517,1
4,500234752,2018-02-11 19:39:53.710,139,0


Making base dataframe

In [42]:
df = pd.merge(sales.drop(['TransactionID'],axis=1),products, on = 'ProductID', how = 'left')
df = pd.merge(df,customers[['UserID','UserFirstTransaction']], on = 'UserID', how = 'left')
df = pd.merge(df,last_transactions.drop(['Churned'],axis=1), on = 'UserID', how = 'left')
df.head(3)

,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount,Category,UserFirstTransaction,LastTransaction,Recency
0,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.0,No,Female Shoes,2015-03-18,2018-06-28 19:35:53.360,2
1,500240813,2017-01-01 03:27:58.490,10610,WEB,Cash,108.0,No,Male Fashion,2015-07-24,2018-06-22 17:42:49.730,8
2,500460527,2017-01-01 04:42:46.500,10579,MOBILE,Cash,22.5,No,Female Fashion,2015-01-06,2018-06-25 01:12:09.600,5


In [43]:
df['SubDiffDay'] = (df['LastTransaction'] - df['UserFirstTransaction']).dt.days
df.head(3)

,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount,Category,UserFirstTransaction,LastTransaction,Recency,SubDiffDay
0,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.0,No,Female Shoes,2015-03-18,2018-06-28 19:35:53.360,2,1198.0
1,500240813,2017-01-01 03:27:58.490,10610,WEB,Cash,108.0,No,Male Fashion,2015-07-24,2018-06-22 17:42:49.730,8,1064.0
2,500460527,2017-01-01 04:42:46.500,10579,MOBILE,Cash,22.5,No,Female Fashion,2015-01-06,2018-06-25 01:12:09.600,5,1266.0


Splitting the data

In [44]:
cut_off = pd.to_datetime('2017-10-01')

In [45]:
Train = df[df['DateTime'] < cut_off]
Test = df[df['DateTime'] >= cut_off]

In [46]:
Train = Train.drop(['ProductID', 'UserFirstTransaction', 'LastTransaction', 'DateTime'], axis = 1)
Test = Test.drop(['ProductID', 'UserFirstTransaction', 'LastTransaction', 'DateTime'], axis = 1)
Train.head(3)

,UserID,Channel,PaymentType,Price,Discount,Category,Recency,SubDiffDay
0,500546547,MOBILE,Cash,51.0,No,Female Shoes,2,1198.0
1,500240813,WEB,Cash,108.0,No,Male Fashion,8,1064.0
2,500460527,MOBILE,Cash,22.5,No,Female Fashion,5,1266.0


In [47]:
categorical_columns = ['Channel','PaymentType','Discount','Category']

In [48]:
def categorical_ohe(df, categorical_columns):
    for col in categorical_columns:
        dummies = pd.get_dummies(df[col], prefix = col)
        
        # concat the dummy columns with the original dataframe
        df = pd.concat([df,dummies], axis = 1)
        
        # drop the original column
        df = df.drop(col, axis = 1)
    return df

Train = categorical_ohe(Train, categorical_columns)
Test = categorical_ohe(Test, categorical_columns)
Train.head()

,UserID,Price,Recency,SubDiffDay,Channel_MOBILE,Channel_WEB,PaymentType_Cash,PaymentType_Mobile Payment,PaymentType_Online Credit Card,Discount_No,...,Category_Indoor Sports,Category_Kitchen Electronics,Category_Male Fashion,Category_Male Shoes,Category_Outdoor Sports,Category_Smart Phones,Category_Smart Watches,Category_Sound Systems,Category_Sport Shoes,Category_TVs and TV Sets
0,500546547,51.0,2,1198.0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,500240813,108.0,8,1064.0,0,1,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,500460527,22.5,5,1266.0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,500338383,42.0,10,1462.0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
4,500591427,59.7,2,1873.0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0


* Kişiyi genel yargılarken bu kısımda gruplama yapmalı ve kişinin yaptığı her bir alış-veriş için değil genel durumu için yargıda bulunmalıyız.  
* Bu durumda kişinin `Price` değişkenini ortalama değil toplam şeklinde almalıyız ki makine öğrenmesi modelimiz diğer, diğer kullanıcılar alışveriş yaptıkça toplam `Price` değişkeni artarken artmayan kullanıcıları daha kolay tespit etsin.  
* Eğer bu kısımda `Price` değişkenini ortalama alırsak 100 defa 10₺ 'lik alış-veriş biri ile 1 defa 10₺ 'lik alış-veriş yapan kişiyi aynı görür.  
* OHEncode edilmiş olanları ise toplayarak gruplandıracağım çünkü kişinin hangi kategoriden kaç kere alışveriş yaptığı o kişinin nasıl biri olduğu hakkında bilgi verir.  

In [49]:
agg_dict = {
    'Price':'sum',
    'Recency':'mean',
    'SubDiffDay':'mean',
    'Channel_MOBILE':'sum',
    'Channel_WEB':'sum',
    'PaymentType_Cash':'sum',
    'PaymentType_Mobile Payment':'sum',
    'PaymentType_Online Credit Card':'sum',
    'Discount_No':'sum',
    'Discount_Yes':'sum',
    'Category_Computers & Laptops':'sum',
    'Category_Electronic Accessories':'sum',
    'Category_Female Fashion':'sum',
    'Category_Female Shoes':'sum',
    'Category_Hobbies':'sum',
    'Category_Indoor Sports':'sum',
    'Category_Kitchen Electronics':'sum',
    'Category_Male Fashion':'sum',
    'Category_Male Shoes':'sum',
    'Category_Outdoor Sports':'sum',
    'Category_Smart Phones':'sum',
    'Category_Smart Watches':'sum',
    'Category_Sound Systems':'sum',
    'Category_Sport Shoes':'sum', 
    'Category_TVs and TV Sets':'sum'
}

In [50]:
def group_dataframe(df, groupby_columns, agg_dict):
    grouped = df.groupby(groupby_columns).agg(agg_dict).reset_index()
    return grouped

grouped_train = group_dataframe(Train, ['UserID'], agg_dict)
grouped_test = group_dataframe(Test, ['UserID'], agg_dict)
grouped_train.head()

,UserID,Price,Recency,SubDiffDay,Channel_MOBILE,Channel_WEB,PaymentType_Cash,PaymentType_Mobile Payment,PaymentType_Online Credit Card,Discount_No,...,Category_Indoor Sports,Category_Kitchen Electronics,Category_Male Fashion,Category_Male Shoes,Category_Outdoor Sports,Category_Smart Phones,Category_Smart Watches,Category_Sound Systems,Category_Sport Shoes,Category_TVs and TV Sets
0,500234730,28.575,517.0,5.0,2,0,2,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,500234752,482.550,139.0,1571.0,7,2,5,0,4,9,...,0,0,0,0,0,4,0,0,1,1
2,500234763,104.550,44.0,269.0,1,1,2,0,0,2,...,0,0,0,0,0,0,0,0,2,0
3,500234829,731.400,18.0,3206.0,15,1,11,4,1,16,...,0,0,0,0,0,4,0,0,0,4
4,500234840,69.000,395.0,0.0,0,2,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0


In [54]:
grouped_test.head()

,UserID,Price,Recency,SubDiffDay,Channel_MOBILE,Channel_WEB,PaymentType_Cash,PaymentType_Mobile Payment,PaymentType_Online Credit Card,Discount_No,...,Category_Indoor Sports,Category_Kitchen Electronics,Category_Male Fashion,Category_Male Shoes,Category_Outdoor Sports,Category_Smart Phones,Category_Smart Watches,Category_Sound Systems,Category_Sport Shoes,Category_TVs and TV Sets
0,500234532,146.70,195.0,2258.0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,500234631,147.00,50.0,34.0,2,2,4,0,0,4,...,0,0,0,0,0,0,1,0,0,0
2,500234642,735.60,13.0,785.0,4,0,0,0,4,4,...,0,0,0,0,0,1,0,0,0,1
3,500234752,443.55,139.0,1571.0,7,1,4,2,2,8,...,0,0,1,4,0,2,0,0,0,1
4,500234763,1183.95,44.0,269.0,10,0,0,10,0,10,...,0,0,0,0,0,0,0,0,3,1


Adding the personal informations

In [55]:
grouped_train = pd.merge(grouped_train, customers.drop('UserFirstTransaction', axis=1), on = 'UserID', how = 'left')
grouped_train = categorical_ohe(grouped_train, ['Gender', 'Location'])
grouped_train.drop('Gender_FEMALE', axis = 1, inplace = True)
grouped_train.head()

,UserID,Price,Recency,SubDiffDay,Channel_MOBILE,Channel_WEB,PaymentType_Cash,PaymentType_Mobile Payment,PaymentType_Online Credit Card,Discount_No,...,Gender_MALE,Location_ADANA,Location_ANKARA,Location_ANTALYA,Location_BURSA,Location_ESKISEHIR,Location_ISTANBUL,Location_IZMIR,Location_KAYSERI,Location_TRABZON
0,500234730,28.575,517.0,5.0,2,0,2,0,0,1,...,1,0,0,0,0,0,1,0,0,0
1,500234752,482.550,139.0,1571.0,7,2,5,0,4,9,...,1,0,0,0,0,0,1,0,0,0
2,500234763,104.550,44.0,269.0,1,1,2,0,0,2,...,1,1,0,0,0,0,0,0,0,0
3,500234829,731.400,18.0,3206.0,15,1,11,4,1,16,...,1,0,0,0,0,1,0,0,0,0
4,500234840,69.000,395.0,0.0,0,2,0,0,2,2,...,0,0,0,0,0,0,0,1,0,0


In [56]:
grouped_test = pd.merge(grouped_test, customers.drop('UserFirstTransaction', axis=1), on = 'UserID', how = 'left')
grouped_test = categorical_ohe(grouped_test, ['Gender', 'Location'])
grouped_test.drop('Gender_FEMALE', axis = 1, inplace = True)
grouped_test.head()

,UserID,Price,Recency,SubDiffDay,Channel_MOBILE,Channel_WEB,PaymentType_Cash,PaymentType_Mobile Payment,PaymentType_Online Credit Card,Discount_No,...,Gender_MALE,Location_ADANA,Location_ANKARA,Location_ANTALYA,Location_BURSA,Location_ESKISEHIR,Location_ISTANBUL,Location_IZMIR,Location_KAYSERI,Location_TRABZON
0,500234532,146.70,195.0,2258.0,0,1,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,500234631,147.00,50.0,34.0,2,2,4,0,0,4,...,0,0,0,0,1,0,0,0,0,0
2,500234642,735.60,13.0,785.0,4,0,0,0,4,4,...,1,0,0,0,0,0,0,1,0,0
3,500234752,443.55,139.0,1571.0,7,1,4,2,2,8,...,1,0,0,0,0,0,1,0,0,0
4,500234763,1183.95,44.0,269.0,10,0,0,10,0,10,...,1,1,0,0,0,0,0,0,0,0


Adding the target feature

In [57]:
grouped_train = pd.merge(grouped_train, last_transactions[['UserID','Churned']], on = 'UserID', how = 'left')
grouped_train.head()

,UserID,Price,Recency,SubDiffDay,Channel_MOBILE,Channel_WEB,PaymentType_Cash,PaymentType_Mobile Payment,PaymentType_Online Credit Card,Discount_No,...,Location_ADANA,Location_ANKARA,Location_ANTALYA,Location_BURSA,Location_ESKISEHIR,Location_ISTANBUL,Location_IZMIR,Location_KAYSERI,Location_TRABZON,Churned
0,500234730,28.575,517.0,5.0,2,0,2,0,0,1,...,0,0,0,0,0,1,0,0,0,1
1,500234752,482.550,139.0,1571.0,7,2,5,0,4,9,...,0,0,0,0,0,1,0,0,0,0
2,500234763,104.550,44.0,269.0,1,1,2,0,0,2,...,1,0,0,0,0,0,0,0,0,0
3,500234829,731.400,18.0,3206.0,15,1,11,4,1,16,...,0,0,0,0,1,0,0,0,0,0
4,500234840,69.000,395.0,0.0,0,2,0,0,2,2,...,0,0,0,0,0,0,1,0,0,1


Check if all customers in generally form

In [58]:
np.sum(grouped_train['UserID'].value_counts()>1)

0

In [60]:
grouped_train.to_csv('../Datasets/ML Datasets/train.csv', index = False)
grouped_test.to_csv('../Datasets/ML Datasets/test.csv', index = False)

<hr>